<a href="https://colab.research.google.com/github/Sandwhaletree/2023.05_Tibame/blob/main/%E9%80%B2%E9%9A%8EDL_%E6%9D%8E%E6%99%BA%E6%8F%9A_Class/005_YOLOv7_BCCD_0718.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### YOLOv7

下載github yolov7 repo

* 官方：https://github.com/WongKinYiu/yolov7.git
* Fork版本有些修改：https://github.com/taipingeric/yolov7.git

In [ ]:
# download github repo
!git clone https://github.com/taipingeric/yolov7.git

#### Download dataset

BCCD: https://public.roboflow.com/object-detection/bccd

In [ ]:
# download Blood Cell dataset
!pip install --upgrade gdown
!gdown --fuzzy '1EjOsWZTtqVCU4ZPDAl5qmuf0EhUMEaWT' -O dataset.zip
# unzip dataset
!unzip -q dataset.zip

#### YOLOv7 支援格式1

*   train
    * images
        * img1.jpg
    * labels
        * img1.txt
*   valid
    * 同train
*   test
    * 同train


txt格式 (YOLOv5)

**class id, x, y, width, height**

x,y,w,h 皆為normalized數值 0~1
```
0 0.0515 0.0251 0.0710 0.0498
1 0.3182 0.0465 0.0735 0.0902
1 0.4197 0.0535 0.0655 0.0997
```

**yolov7/data/coco.yaml**
```yaml
train: ../train/images # 訓練圖片資料夾
val: ../valid/images # 驗證圖片資料夾
test: ../test/images # 測試圖片資料夾 (Optional)

# number of classes
nc: 3 # 類別數

# class names
names: ['Platelets', 'RBC', 'WBC']
```

#### YOLOv7 支援格式2


/coco128
*   images
    * train2017
        * img1.jpg
        * img2.jpg
    * val2017
        * img3.jpg
*   labels
    * train2017
        * img1.txt
        * img2.txt
    * val2017
        * img3.txt


txt格式 (YOLOv5)

**class id, x, y, width, height**

x,y,w,h 皆為normalized數值 0~1
```
45 0.479492 0.688771 0.955609 0.5955
45 0.736516 0.247188 0.498875 0.476417
50 0.637063 0.732938 0.494125 0.510583
45 0.339438 0.418896 0.678875 0.7815
49 0.646836 0.132552 0.118047 0.096937
49 0.773148 0.129802 0.090734 0.097229
49 0.668297 0.226906 0.131281 0.146896
49 0.642859 0.079219 0.148063 0.148062
```

#### Training

In [ ]:
!pwd

In [ ]:
# 切換至yolov7 repo
%cd yolov7

In [ ]:
!ls

In [ ]:
# 安裝套件
!pip install -r requirements.txt

# 修改config檔



1.   data config: **data/coco.yaml** (資料設定檔)
    1.   刪除 ```download: bash ./scripts/get_coco.sh``` 避免下載COCO資料集
    2.   將train: `./coco/train2017.txt` 替換成訓練的圖片資料夾
    3.   nc: 80 替換成類別數 (BCCD: 3)
    4.   names: 替換成類別名稱['WBC', 'RBC', 'Platelets']
    5. 存檔

```yaml
train: ../train/images # 訓練圖片資料夾
val: ../valid/images # 驗證圖片資料夾
test: ../test/images # 測試圖片資料夾 (Optional)

# number of classes
nc: 3 # 類別數

# class names
names: ['Platelets', 'RBC', 'WBC',]
```

2.   model config: **cfg/training/yolov7.yaml** (模型設定檔)
    1. nc: 80  替換成類別數
    2. 存檔



#### Training

--data: 資料集設定檔

--img: img size

--cfg: 模型架構 & 設定

--weight: 初始參數

--name: 實驗名稱

--hyp：超參數

--workers: maximum number of dataloader workers

In [ ]:
!python train.py --workers 8 --device 0 --batch-size 8 --data data/coco.yaml --img 640 640 --cfg cfg/training/yolov7.yaml --weights '' --name yolov7 --hyp data/hyp.scratch.p5.yaml

在 runs/train/實驗名稱 中會存放每次訓練的結果與參數檔案

#### Transfer learning

使用官方COCO pre-trained model做finetune

In [ ]:
!pwd

In [ ]:
# 下載官方pre-trained參數
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [ ]:
# finetune p5 models
!python train.py --workers 8 --device 0 --batch-size 8 --data data/coco.yaml --img 640 640 --cfg cfg/training/yolov7.yaml --weights 'yolov7.pt' --name yolov7-transfer --hyp data/hyp.scratch.custom.yaml

# finetune p6 models
# python train_aux.py --workers 8 --device 0 --batch-size 16 --data data/custom.yaml --img 1280 1280 --cfg cfg/training/yolov7-w6-custom.yaml --weights 'yolov7-w6_training.pt' --name yolov7-w6-custom --hyp data/hyp.scratch.custom.yaml

#### Testing

In [ ]:
# --weights: 測試參數檔案
!python test.py --task "test" --data data/coco.yaml --img 640 --batch 8 --conf 0.001 --iou 0.65 --device 0 --weights "/content/yolov7/runs/train/yolov7-transfer/weights/best.pt" --name yolov7_640_val

#### Inference

下載官方參數檔

In [ ]:
# !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

--save-txt: 輸出txt結果

--save-conf: 在txt中輸出box confidence

In [ ]:
# !python detect.py --weights "/content/yolov7/runs/train/yolov7-transfer/weights/best.pt" --conf 0.25 --img-size 640 --source '/content/test/images/BloodImage_00044_jpg.rf.589ee3d351cb6d9a3f7b7a942da5370a.jpg' --save-txt --save-conf
!python detect.py --weights yolov7.pt --conf 0.25 --img-size 640 --source inference/images/horses.jpg --save-txt --save-conf

在runs/detect/exp*/  看照片偵測後的成果

#### More configs

cfg/training/yolov7.yaml

```yaml
# parameters
nc: 80  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [12,16, 19,36, 40,28]  # P3/8
  - [36,75, 76,55, 72,146]  # P4/16
  - [142,110, 192,243, 459,401]  # P5/32

# yolov7 backbone
# Conv: ch_out, kernel, stride, padding, groups
backbone:
  # [from, number, module, args]
  # Stage 0
  [[-1, 1, Conv, [32, 3, 1]],  # 0
   # Stage 1
   [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2
   [-1, 1, Conv, [64, 3, 1]],
   # Stage 2
   [-1, 1, Conv, [128, 3, 2]],  # 3-P2/4
   # ELAN, 256
   [-1, 1, Conv, [64, 1, 1]],  # -6
   [-2, 1, Conv, [64, 1, 1]],  # -5
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],  # -3
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],  # -1
   [[-1, -3, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1]],  # 11
   # Stage 3
   # Down c_out = 256
   [-1, 1, MP, []], # MP: MaxPooling
   [-1, 1, Conv, [128, 1, 1]],
   [-3, 1, Conv, [128, 1, 1]],
   [-1, 1, Conv, [128, 3, 2]],
   [[-1, -3], 1, Concat, [1]],  # 16-P3/8
   # ELAN, 512
   [-1, 1, Conv, [128, 1, 1]],
   [-2, 1, Conv, [128, 1, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [[-1, -3, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [512, 1, 1]],  # 24
   # Stage 4
   [-1, 1, MP, []],
   [-1, 1, Conv, [256, 1, 1]],
   [-3, 1, Conv, [256, 1, 1]],
   [-1, 1, Conv, [256, 3, 2]],
   [[-1, -3], 1, Concat, [1]],  # 29-P4/16
   # ELAN, 1024
   [-1, 1, Conv, [256, 1, 1]],
   [-2, 1, Conv, [256, 1, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [[-1, -3, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [1024, 1, 1]],  # 37
   # Stage 5
   [-1, 1, MP, []],
   [-1, 1, Conv, [512, 1, 1]],
   [-3, 1, Conv, [512, 1, 1]],
   [-1, 1, Conv, [512, 3, 2]],
   [[-1, -3], 1, Concat, [1]],  # 42-P5/32
   # ELAN, 1024
   [-1, 1, Conv, [256, 1, 1]],
   [-2, 1, Conv, [256, 1, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [[-1, -3, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [1024, 1, 1]],  # 50
  ]

# yolov7 head
head:
  # Stage: 5
  [[-1, 1, SPPCSPC, [512]], # 51
   # Stage 4
   # *2 Up, 512
   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [37, 1, Conv, [256, 1, 1]], # route backbone P4
   [[-1, -2], 1, Concat, [1]],
   # ELAN, 256
   [-1, 1, Conv, [256, 1, 1]], # -6
   [-2, 1, Conv, [256, 1, 1]], # -5
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1]], # 63
   # Stage 3
   # *2 Up, 512
   [-1, 1, Conv, [128, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [24, 1, Conv, [128, 1, 1]], # route backbone P3
   [[-1, -2], 1, Concat, [1]],
   # ELAN, 128
   [-1, 1, Conv, [128, 1, 1]],
   [-2, 1, Conv, [128, 1, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [128, 1, 1]], # 75
   # Stage 4
   # Down, 512
   [-1, 1, MP, []],
   [-1, 1, Conv, [128, 1, 1]],
   [-3, 1, Conv, [128, 1, 1]],
   [-1, 1, Conv, [128, 3, 2]],
   [[-1, -3, 63], 1, Concat, [1]],
   # ELAN, 256
   [-1, 1, Conv, [256, 1, 1]],
   [-2, 1, Conv, [256, 1, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1]], # 88
   # Stage 5
   # Down, 1024
   [-1, 1, MP, []],
   [-1, 1, Conv, [256, 1, 1]],
   [-3, 1, Conv, [256, 1, 1]],
   [-1, 1, Conv, [256, 3, 2]],
   [[-1, -3, 51], 1, Concat, [1]],
   # ELAN, 512
   [-1, 1, Conv, [512, 1, 1]],
   [-2, 1, Conv, [512, 1, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [512, 1, 1]], # 101

   [75, 1, RepConv, [256, 3, 1]],
   [88, 1, RepConv, [512, 3, 1]],
   [101, 1, RepConv, [1024, 3, 1]],

   [[102,103,104], 1, IDetect, [nc, anchors]],   # Detect(P3, P4, P5)
  ]

```

#### Important Files

yolov7



*   train.py: 訓練script
*   test.py: 測試script
* detect.py: 偵測圖片或影片
* cfg/: 模型架構設定檔
* data/: 資料集設定檔
    * coco.yaml: 範例格式
    * hyp.*.yaml: 超參數設定
* models/: 模型程式碼
    * yolo.py
        * parse_model: yaml config轉模型
    * common.py: 網路層
* paper/: 論文pdf (含附錄)
* tools/: demo 筆記本檔
* utils/: 其他程式碼
